## Initialise

In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.container {
    width: 100%;
}
</style>
""")

In [2]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import tools 
import initialise
import ensemble as en

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.
2016-03-21 12:43:38,137 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458524616.log


This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.


In [12]:
def model_cv(data):
    
    return en.Ensemble([gl.boosted_trees_classifier.create(train, validation_set=validate, **params) for 
                train, validate in gl.cross_validation.KFold(data, 5)])


In [3]:
train_data, test_data = initialise.load_data(reload_data=False)

Loading saved processed data
284  features in total


In [16]:
params = {'target':'TARGET', 'random_seed':8923, 'verbose':False,
          'class_weights':'auto','metric':'auc',
          'early_stopping_rounds':10, 'max_iterations':500,
          'step_size': 0.1,
          'max_depth': 5,
          'column_subsample': 0.6,
          'row_subsample': 0.95}

models = model_cv(train_data)
models

<ensemble.Ensemble instance at 0x7fb2a40cad40>

## Testing basic ensemble module

In [17]:
models.predict(test_data[0:10], output_type = 'probability')

dtype: float
Rows: 10
[0.4538480281829835, 0.3836626470088959, 0.9368736863136292, 0.7336451053619385, 0.922076463699341, 0.16256090700626372, 0.5547891736030579, 0.1972264736890793, 0.5734805941581727, 0.6654268145561218]

In [19]:
models.predict(test_data[0:10], output_type = 'class')

dtype: str
Rows: 10
['Satisified', 'Satisified', 'Unsatisified', 'Unsatisified', 'Unsatisified', 'Satisified', 'Unsatisified', 'Satisified', 'Unsatisified', 'Unsatisified']

In [103]:
models2.vote_fn=en.vote
print models2

{'vote_fn': <function vote at 0x7fb294e8c2a8>, 'weights': [0.2, 0.2, 0.2, 0.2, 0.2]}
['boosted_trees_classifier', 'boosted_trees_classifier', 'boosted_trees_classifier', 'boosted_trees_classifier', 'boosted_trees_classifier']


In [9]:
reload(en)
models2 =  en.load_model('Models/ensemble_5fold')
print models2

{'vote_fn': u'vote', 'weights': [0.2, 0.2, 0.2, 0.2, 0.2]}
['boosted_trees_classifier', 'boosted_trees_classifier', 'boosted_trees_classifier', 'boosted_trees_classifier', 'boosted_trees_classifier']


In [10]:

models2.evaluate(train_data, metric='auc')

no weights


{'auc': 0.8655960176795484}

In [11]:
models2.evaluate(train_data, metric='auc')

no weights


{'auc': 0.8655960176795484}

In [12]:
models2.predict(train_data[0:4], weights=[0.1, 0.9])

dtype: float
Rows: 4
[0.5619310170412064, 0.7661639094352722, 0.9230569243431092, 0.5932426571846008]

In [13]:
models2.predict(train_data[0:4], weights=[0.9, 0.1])

dtype: float
Rows: 4
[0.4957584410905838, 0.7420456767082215, 0.8939605116844177, 0.6221562504768372]

In [14]:
models2.evaluate(train_data, metric='auc', weights=[.1, .9])

{'auc': 0.8632903839546622}

In [15]:
models2.evaluate(train_data, metric='auc', weights=[.9, 0.1])

{'auc': 0.8644038863365227}

# GBT + RF ensemble

In [17]:
params_GBT = {'target':'TARGET', 'random_seed':8923, 'verbose':False,
              'class_weights':'auto','metric':'auc',
              'early_stopping_rounds':10, 'max_iterations':500,
              'step_size': 0.1, 'max_depth': 5, 'column_subsample': 0.6, 'row_subsample': 0.95}
model_GBT = gl.boosted_trees_classifier.create(train_data, **params_GBT)
model_GBT.evaluate(train_data, metric='auc')

{'auc': 0.8732945299322691}

In [34]:
params_RF = {'target':'TARGET', 'random_seed':8923, 'verbose':True,
          'class_weights':'auto','metric':'auc',
            'num_trees':100}

#model_RF = gl.random_forest_classifier.create(train_data, validation_set=validate, **params)

In [35]:
train, validate = train_data.random_split(0.8, seed=8923)
model_RF = gl.random_forest_classifier.create(train, validation_set=validate, **params_RF)

WARNING: Detected extremely low variance for feature(s) 'var18 indicator 0', 'var18 number 0', 'var18 amortization amount delta 1Y3', 'var33 delta transfer amount out 1Y3', 'var18 repayment amount ultima1', 'var33 transfer amount out ultima1' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Random forest classifier:

--------------------------------------------------------

Number of examples          : 60827

Number of classes           : 2

...

| 85        | 29.845226    | 0.859642     | 0.830548       |

| 90        | 31.616770    | 0.859559     | 0.830519       |

| 95        | 33.369113    | 0.859515     | 0.830558       |

| 100       | 35.111622    | 0.859409     | 0.830279       |

+-----------+--------------+--------------+----------------+

In [36]:
print model_RF.evaluate(train_data, metric='auc'), model_GBT.evaluate(train_data, metric='auc')

{'auc': 0.8534246406121243} {'auc': 0.8732945299322691}


In [37]:
models = en.Ensemble([model_GBT,model_RF], weights=[0.9, 0.1])

In [38]:
models.weights

[0.9, 0.1]

In [39]:
models.evaluate(train_data, metric='auc')

no weights


{'auc': 0.8723899838382745}

In [42]:
tools.make_submission(models, test_data, 'en_GBT_RF')

no weights
